In [1]:
import tensorflow as tf
import numpy as np
from BregmanToolkit.bregman.suite import *

ImportError: No module named 'BregmanToolkit'

In [2]:
filenames = tf.train.match_filenames_once('./audio_dataset/*.wav')
count_num_files = tf.size(filenames)
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.WholeFileReader()
filename, file_contents = reader.read(filename_queue)

chroma = tf.placeholder(tf.float32) # 가장 큰 기여를 하는 음고를 식별하기 위한 연산을 생성합니다
max_freqs = tf.argmax(chroma, 0)

In [3]:
def get_next_chromagram(sess):
    audio_file = sess.run(filename)
    F = Chromagram(audio_file, nfft=16384, wfft=8192, nhop=2205)
    return F.X

def extract_feature_vector(sess, chroma_data): # 크로마그램을 피쳐 벡터로 변환합니다
    num_features, num_samples = np.shape(chroma_data)
    freq_vals = sess.run(max_freqs, feed_dict={chroma: chroma_data})
    hist, bins = np.histogram(freq_vals, bins=range(num_features + 1))
    return hist.astype(float) / num_samples

def get_dataset(sess): # 각 행이 데이터 아이템인 행렬을 구성합니다
    num_files = sess.run(count_num_files)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    xs = []
    for _ in range(num_files):
        chroma_data = get_next_chromagram(sess)
        x = [extract_feature_vector(sess, chroma_data)]
        x = np.matrix(x)
        if len(xs) == 0:
            xs = x 
        else:
            xs = np.vstack((xs, x))
    return xs